## Импорт библиотек


In [8]:
import swifter
import numpy as np
import pandas as pd
import pyarrow as pa

import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords, wordnet
from nltk.tokenize import regexp_tokenize
from nltk.tag import pos_tag
from textblob import TextBlob
from difflib import SequenceMatcher

from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.pipeline import Pipeline
from tqdm import tqdm

In [9]:
%config InlineBackend.figure_format = 'retina'

RND = 322
STOPWORDS = set(stopwords.words())
PATTERN = r"[a-zA-Z]+(?:[-'.@&/][a-zA-Z]+)*"

tqdm.pandas()
nltk.download(["punkt", "wordnet", "stopwords"]);

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Vovan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Vovan\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Vovan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Подготовка данных


Предобработку текста будем осуществлять с помощью пайплайна, создав предварительно 4 класса:

- TextPreprocessor - создаёт признаки с помощью _nltk_;
- TextFeaturesExtractor - создаёт признаки с помощью _pandas_ и _difflib_;
- TextBlobFeaturesExtractor - создаёт призкаки средствами библиотеки _textblob_;
- DtypeOptimizer - оптимизация типов данных.


In [10]:
class TextPreprocessor(BaseEstimator, TransformerMixin):
    """Создание новых признаков из текстовых данных с помощью библиотеки NLTK"""

    def __init__(self, pattern: str = PATTERN, stopwords: set = STOPWORDS):
        self.pattern = pattern
        self.stopwords = stopwords

    def tag_words_with_pos(self, words: list) -> list:
        """
        Присваивает теги словам на основе их частей речи.

        Параметры:
            - words (list): Список слов.

        Возвращает:
            - list: Список кортежей, содержащих слова и соответствующие теги.
        """
        pos_dict = {
            "J": wordnet.ADJ,
            "V": wordnet.VERB,
            "N": wordnet.NOUN,
            "R": wordnet.ADV,
        }

        tagged_words = pos_tag(words)
        tagged_words_with_pos = [
            (word, pos_dict.get(tag[0], wordnet.NOUN)) for (word, tag) in tagged_words
        ]

        return tagged_words_with_pos

    def filter_words(self, words: list) -> list:
        """
        Фильтрует список слов, удаляя стоп-слова.

        Параметры:
            - words (list): Список слов.

        Возвращает:
            - list: Список отфильтрованных слов.
        """

        filtered_words = [word for word in words if word not in self.stopwords]
        return filtered_words

    def lemmatize_words(self, words: list) -> list:
        """
        Приводит слова к их базовым формам (леммам) с помощью лемматизации.

        Параметры:
            - words (list): Список слов.

        Возвращает:
            - list: Список лемматизированных слов.
        """

        wnl = WordNetLemmatizer()
        lemmatized_words = [wnl.lemmatize(word, tag) for word, tag in words]
        return lemmatized_words

    def fit(self, X, y=None):
        return self

    def transform(self, X: pd.DataFrame) -> pd.DataFrame:

        X = X.copy()

        # Проводим токенизацию и удаление стоп слов
        posts_tokenized = (
            X.loc[::5, "post_text_fix"]
            .swifter.apply(regexp_tokenize, pattern=PATTERN)
            .values
        )

        X["post_text_reg_tok"] = np.repeat(posts_tokenized, 5)

        posts_tok_clr = (
            X.loc[::5, "post_text_reg_tok"].swifter.apply(
                self.filter_words).values
        )

        X["post_text_tok_clr"] = np.repeat(posts_tok_clr, 5)

        X["comment_text_reg_tok"] = X["comment_text_fix"].swifter.apply(
            regexp_tokenize, pattern=PATTERN
        )

        X["comment_text_tok_clr"] = X["comment_text_reg_tok"].swifter.apply(
            self.filter_words
        )

        # Простановка частей речи
        posts_text_tcm = (
            X.loc[::5, "post_text_tok_clr"]
            .swifter.apply(self.tag_words_with_pos)
            .values
        )
        X["post_text_tcm"] = np.repeat(posts_text_tcm, 5)
        X["comment_text_tcm"] = X["comment_text_tok_clr"].swifter.apply(
            self.tag_words_with_pos
        )

        # Лемматизация текста

        post_text_lemmatized = (
            X.loc[::5, "post_text_tcm"].swifter.apply(
                self.lemmatize_words).values
        )

        X["post_text_lemmatized"] = np.repeat(post_text_lemmatized, 5)
        X["comment_text_lemmatized"] = X["comment_text_tcm"].swifter.apply(
            self.lemmatize_words
        )

        return X


class TextFeaturesExtractor(BaseEstimator, TransformerMixin):
    """Создание новых признаков из текстовых данных с помощью библиотеки pandas и difflib
    """

    def sequence_matcher(self, row: pd.Series) -> float:
        """
        Вычисляет косинусное сходство (cosine similarity) между двумя текстовыми строками с использованием SequenceMatcher.

        Параметры:
            - row (pd.Series): Строка данных, содержащая текстовые поля "post_text_fix" и "comment_text_fix".

        Возвращает:
            - float: Значение косинусного сходства между строками.
        """
        cos_sim = SequenceMatcher(
            a=row["post_text_fix"],
            b=row["comment_text_fix"],
        ).ratio()
        return cos_sim

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        # Определение длины строки
        X["post_text_len"] = X["post_text_fix"].str.len()
        X["comment_text_len"] = X["comment_text_fix"].str.len()
        # Определение количества слов
        X["post_text_words_count"] = X["post_text_reg_tok"].apply(len)
        X["comment_text_words_count"] = X["comment_text_reg_tok"].apply(len)
        # Определение количества ссылок в строке
        X["post_text_links_count"] = X["post_text_fix"].str.count(r"\burl\b")
        X["comment_text_links_count"] = X["comment_text_fix"].str.count(
            r"\burl\b")
        # Определение косинусного сходства между текстом поста и комментария
        X["cosine_similarity"] = X.swifter.apply(self.sequence_matcher, axis=1)

        return X


class TextBlobFeaturesExtractor(BaseEstimator, TransformerMixin):
    """Создание новых признаков из текстовых данных с помощью библиотеки textblob
    """

    def text_polarity(self, text: str) -> tuple(float, float, int):
        """
        Вычисляет количество предложений, полярность и субъективность текста с использованием TextBlob.

        Параметры:
            - text (str): Текст для анализа.

        Возвращает:
            - Tuple[float, float, int]: Кортеж с полярностью, субъективностью и количеством предложений.
        """
        blob = TextBlob(text)
        polarity = blob.sentiment.polarity
        subjectivity = blob.sentiment.subjectivity
        line_count = len(blob.sentences)
        return polarity, subjectivity, line_count

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()

        # Добавляем колонки для комментариев
        (
            X["comment_text_polarity"],
            X["comment_text_subjectivity"],
            X["comment_text_line_count"],
        ) = zip(*X["comment_text_fix"].swifter.apply(self.text_polarity))

        # Добавляем колонки для постов
        (
            X["post_text_polarity"],
            X["post_text_subjectivity"],
            X["post_text_line_count"],
        ) = zip(*X["post_text_fix"].swifter.apply(self.text_polarity))

        return X


class DtypeOptimizer(BaseEstimator, TransformerMixin):
    """Оптимизация целочисленных типов данных в pd.DataFrame"""

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        for col in X.select_dtypes("int").columns:
            min_val = X[col].min()
            max_val = X[col].max()
            # проверка беззнаковых типов
            if min_val >= 0:
                if max_val < np.iinfo(np.uint8).max:
                    X[col] = X[col].astype("uint8")
                elif max_val < np.iinfo(np.uint16).max:
                    X[col] = X[col].astype("uint16")
                elif max_val < np.iinfo(np.uint32).max:
                    X[col] = X[col].astype("uint32")

            elif min_val >= np.iinfo(np.int8).min and max_val < np.iinfo(np.int8).max:
                X[col] = X[col].astype("int8")
            elif min_val >= np.iinfo(np.int16).min and max_val < np.iinfo(np.int16).max:
                X[col] = X[col].astype("int16")
            elif (
                min_val >= -
                    np.iinfo(np.int32).min and max_val < np.iinfo(np.int32).max
            ):
                X[col] = X[col].astype("int32")
        return X

In [11]:
pipe = Pipeline(
    [
        ("nltk", TextPreprocessor()),
        ("pandas_str", TextFeaturesExtractor()),
        ("text_blob", TextBlobFeaturesExtractor()),
        ("opt_dtypes", DtypeOptimizer()),
    ]
)


Подгружаем предварительно очищенные данные


In [12]:
df_train = pd.read_feather("data/df_train_BERT.feather")
df_test = pd.read_feather("data/df_test_BERT.feather")
df_train.info()
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 440535 entries, 0 to 440534
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   post_index        440535 non-null  int64 
 1   post_text         440535 non-null  object
 2   comment_text      440535 non-null  object
 3   comment_score     440535 non-null  int64 
 4   post_text_fix     440535 non-null  object
 5   comment_text_fix  440535 non-null  object
dtypes: int64(2), object(4)
memory usage: 20.2+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70020 entries, 0 to 70019
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   post_index        70020 non-null  int64 
 1   post_text         70020 non-null  object
 2   comment_text      70020 non-null  object
 3   comment_score     0 non-null      object
 4   post_text_fix     70020 non-null  object
 5   comment_text_fix  70020 non-

In [13]:
display(df_train.head(2))
display(df_test.head(2))

,post_index,post_text,comment_text,comment_score,post_text_fix,comment_text_fix
0,0,"iOS 8.0.1 released, broken on iPhone 6 models,...",I am still waiting for them to stabilize wifi ...,None,"ios 8.0.1 released, broken on iphone 6 models,...",i am still waiting for them to stabilize wifi ...
1,0,"iOS 8.0.1 released, broken on iPhone 6 models,...","For those who upgraded, no need to do a restor...",None,"ios 8.0.1 released, broken on iphone 6 models,...","for those who upgraded, no need to do a restor..."


Применяем обработку текста и сохраняем данные для обучающей и тестовой выборки в файлы


In [ ]:
%%time
df_train_transformed = pipe.fit_transform(df_train)

In [15]:
df_train_transformed.to_parquet(
    "data/df_train_with_features.parquet", engine="pyarrow"
)

CPU times: total: 1min 13s
Wall time: 1min 7s


In [ ]:
%%time
df_test_transformed = pipe.transform(df_test)

In [17]:
%%time
df_test_transformed.to_parquet(
    "data/df_test_with_features.parquet", engine="pyarrow")

CPU times: total: 20.5 s
Wall time: 18.6 s
